In [55]:
!pip install casadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
!pip install progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import copy
from casadi import *


class ODEModel:
    """
    This class creates an ODE model using casadi symbolic framework
    """

    def __init__(self, dt, x, dx, J=None, y=None, u=None, d=None, p=None):
        self.dt = dt  # sampling
        self.x = x  # states (sym)
        self.y = MX.sym('y', 0) if y is None else y  # outputs (sym)
        self.u = MX.sym('u', 0) if u is None else u  # inputs (sym)
        self.d = MX.sym('d', 0) if d is None else d  # disturbances (sym)
        self.p = MX.sym('p', 0) if p is None else p  # parameters (sym)
        self.J = MX.sym('J', 0) if J is None else p  # cost function
        self.dx = dx  # model equations
        #self.theta = vertcat(self.d, self.p)  # parameters to be estimated vector (sym)

    def get_equations(self, intg='idas'):
        """
        Gets equations and integrator
        """

        self.ode = {
            'x': self.x,
            'p': vertcat(self.u, self.d, self.p),
            'ode': self.dx, 
            'quad': self.J
        }  # ODE model

        self.F = Function('F', [self.x, self.u, self.d, self.p], [self.dx, self.J, self.y],
                          ['x', 'u', 'd', 'p'], ['dx', 'J', 'y'])  # model function
        self.rfsolver = rootfinder('rfsolver', 'newton', self.F)  # rootfinder
        self.opts = {'tf': self.dt}  # sampling time
        self.Plant = integrator('F', intg, self.ode, self.opts)  # integrator

    def steady(self, xguess=None, uf=None, df=None, pf=None):
        """
        Calculates root
        """

        xguess = np.zeros(self.x.shape[0]) if xguess is None else xguess
        uf = [] if uf is None else uf
        df = [] if df is None else df
        pf = [] if pf is None else pf
        sol = self.rfsolver(x=xguess, u=uf, d=df, p=pf)
        return {
            'x': sol['y'].full(),
            'J': sol['J'].full()
        }

    def simulate_step(self, xf, uf=None, df=None, pf=None):
        """
        Simulates 1 step
        """

        uf = [] if uf is None else uf
        df = [] if df is None else df
        pf = [] if pf is None else pf
        Fk = self.Plant(x0=xf, p=vertcat(uf, df, pf))  # integration

        return {
            'x': Fk['xf'].full().reshape(-1),
            'u': uf, 
            'd': df,
            'p': pf
        }

    def check_steady(self, nss, t, cov, ysim):
        """
        Steady-state identification
        """

        s2 = [0] * len(cov)
        M = np.mean(ysim[t - nss:t, :], axis=0)
        for i in range(0, nss):
            s2 += np.power(ysim[t - i - 1, :] - M, 2)
        S2 = s2 / (nss - 1)
        if np.all(S2 <= cov):
            flag = True
        else:
            flag = False
        return {
            'Status': flag,
            'S2': S2
        }

    def build_nlp_steady(self, xguess=None, uguess=None, lbx=None, ubx=None,
                         lbu=None, ubu=None, opts={}):
        """
        Builds steady-state optimization NLP
        """
        # Guesses and bounds
        xguess = np.zeros(self.x.shape[0]) if xguess is None else xguess
        uguess = np.zeros(self.u.shape[0]) if uguess is None else uguess
        lbx = -inf * np.ones(self.x.shape[0]) if lbx is None else lbx
        lbu = -inf * np.ones(self.u.shape[0]) if lbu is None else lbu
        ubx = +inf * np.ones(self.x.shape[0]) if ubx is None else ubx
        ubu = +inf * np.ones(self.u.shape[0]) if ubu is None else ubu

        # Removing Nones inside vectors
        if None in xguess: xguess = np.array([0 if v is None else v for v in xguess])
        if None in uguess: uguess = np.array([0 if v is None else v for v in uguess])
        if None in lbx: lbx = np.array([-inf if v is None else v for v in lbx])
        if None in lbu: lbu = np.array([-inf if v is None else v for v in lbu])
        if None in ubx: ubx = np.array([+inf if v is None else v for v in ubx])
        if None in ubu: ubu = np.array([+inf if v is None else v for v in ubu])

        # Empty NLP
        self.w = []
        self.w0 = []
        self.lbw = []
        self.ubw = []
        self.g = []
        self.lbg = []
        self.ubg = []

        # Start NLP
        self.w += [self.x, self.u]
        self.w0 += list(xguess)
        self.w0 += list(uguess)
        self.lbw += list(lbx)
        self.lbw += list(lbu)
        self.ubw += list(ubx)
        self.ubw += list(ubu)
        self.g += [vertcat(self.dx)]
        self.lbg += list(np.zeros(self.dx.shape[0]))
        self.ubg += list(np.zeros(self.dx.shape[0]))

        nlp = {
            'x': vertcat(*self.w),
            'p': vertcat(self.d, self.p),
            'f': self.J,
            'g': vertcat(*self.g)
        }

        # Solver
        self.solver = nlpsol('solver', 'ipopt', nlp, opts)

    def optimize_steady(self, ksim=None, df=[], pf=[]):
        """
        Performs 1 optimization step (df and pf must be lists)
        """

        # Solver run
        sol = self.solver(x0=vertcat(*self.w0), p=vertcat(df+pf),
                          lbx=vertcat(*self.lbw), ubx=vertcat(*self.ubw),
                          lbg=vertcat(*self.lbg), ubg=vertcat(*self.ubg))
        flag = self.solver.stats()

        if ksim != None:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if optimization converged
                print('Optimization step ' + str(ksim) + ': Solver did not converge.')
            else:
                print('Optimization step ' + str(ksim) + ': Optimal Solution Found.')
        else:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if optimization converged
                print('Optimization step: Solver did not converge.')
            else:
                print('Optimization step: Optimal Solution Found.')

                # Solution
        wopt = sol['x'].full()  # solution
        self.w0 = copy.deepcopy(wopt)  # solution as guess for the next opt step
        return {
            'x': wopt[:self.x.shape[0]],
            'u': wopt[-self.u.shape[0]:]
        }

    def build_nlp_dyn(self, N, M, xguess, uguess, lbx=None, ubx=None, lbu=None,
                      ubu=None, m=3, pol='legendre', opts={}):
        """
        Build dynamic optimization NLP
        """

        self.m = m
        self.N = N
        self.M = M
        self.pol = pol

        # Guesses and bounds
        xguess = np.zeros(self.x.shape[0]) if xguess is None else xguess
        uguess = np.zeros(self.u.shape[0]) if uguess is None else uguess
        lbx = -inf*np.ones(self.x.shape[0]) if lbx is None else lbx
        lbu = -inf*np.ones(self.u.shape[0]) if lbu is None else lbu
        ubx = +inf*np.ones(self.x.shape[0]) if ubx is None else ubx
        ubu = +inf*np.ones(self.u.shape[0]) if ubu is None else ubu

        # Removing Nones inside vectors
        if None in xguess: xguess = np.array([0 if v is None else v for v in xguess])
        if None in uguess: uguess = np.array([0 if v is None else v for v in uguess])
        if None in lbx: lbx = np.array([-inf if v is None else v for v in lbx])
        if None in lbu: lbu = np.array([-inf if v is None else v for v in lbu])
        if None in ubx: ubx = np.array([+inf if v is None else v for v in ubx])
        if None in ubu: ubu = np.array([+inf if v is None else v for v in ubu])

        # Polynomials
        self.tau = np.array([0] + collocation_points(self.m, self.pol))
        self.L = np.zeros((self.m + 1, 1))
        self.Ldot = np.zeros((self.m + 1, self.m + 1))
        self.Lint = self.L
        for i in range(0, self.m + 1):
            coeff = 1
            for j in range(0, self.m + 1):
                if j != i:
                    coeff = np.convolve(coeff, [1, -self.tau[j]]) / \
                            (self.tau[i] - self.tau[j])
            self.L[i] = np.polyval(coeff, 1)
            ldot = np.polyder(coeff)
            for j in range(0, self.m + 1):
                self.Ldot[i, j] = np.polyval(ldot, self.tau[j])
            lint = np.polyint(coeff)
            self.Lint[i] = np.polyval(lint, 1)

        # "Lift" initial conditions
        xk = MX.sym('x0', self.x.shape[0])  # first point at each interval
        x0_sym = MX.sym('x0_par', self.x.shape[0])  # first point
        uk_prev = uguess

        # Empty NLP
        self.w = []
        self.w0 = []
        self.lbw = []
        self.ubw = []
        self.g = []
        self.lbg = []
        self.ubg = []
        self.J = 0

        # Start NLP
        self.w += [xk]
        self.w0 += list(xguess)
        self.lbw += list(lbx)
        self.ubw += list(ubx)
        self.g += [xk - x0_sym]
        self.lbg += list(np.zeros(self.dx.shape[0]))
        self.ubg += list(np.zeros(self.dx.shape[0]))

        # NLP build
        for k in range(0, self.N):
            xki = []  # state at collocation points
            for i in range(0, self.m):
                xki.append(MX.sym('x_' + str(k + 1) + '_' + str(i + 1), self.x.shape[0]))
                self.w += [xki[i]]
                self.lbw += list(lbx)
                self.ubw += list(ubx)
                self.w0 += list(xguess)

            # uk as decision variable
            uk = MX.sym('u_' + str(k + 1), self.u.shape[0])
            self.w += [uk]
            self.lbw += list(lbu)
            self.ubw += list(ubu)
            self.w0 += list(uguess)

            if k >= self.M:
                self.g += [uk - uk_prev]  # delta_u
                self.lbg += list(np.zeros(self.u.shape[0]))
                self.ubg += list(np.zeros(self.u.shape[0]))

            uk_prev = uk

            # Loop over collocation points
            xk_end = self.L[0] * xk
            for i in range(0, self.m):
                xk_end += self.L[i + 1] * xki[i]  # add contribution to the end state
                xc = self.Ldot[0, i + 1] * xk  # expression for the state derivative at the collocation poin
                for j in range(0, m):
                    xc += self.Ldot[j + 1, i + 1] * xki[j]
                fi = self.F(xki[i], uk, self.d, self.p)  # model and cost function
                self.g += [self.dt * fi[0] - xc]  # model equality contraints reformulated
                self.lbg += list(np.zeros(self.x.shape[0]))
                self.ubg += list(np.zeros(self.x.shape[0]))
                # self.J += self.dt*fi[1]*self.Lint[i+1] #add contribution to obj. quadrature function

            # New NLP variable for state at end of interval
            xk = MX.sym('x_' + str(k + 2), self.x.shape[0])
            self.w += [xk]
            self.lbw += list(lbx)
            self.ubw += list(ubx)
            self.w0 += list(xguess)

            # No shooting-gap constraint
            self.g += [xk - xk_end]
            self.lbg += list(np.zeros(self.x.shape[0]))
            self.ubg += list(np.zeros(self.x.shape[0]))

        self.J = fi[1]

        # NLP
        self.nlp = {
            'x': vertcat(*self.w),
            'f': self.J,
            'g': vertcat(*self.g),
            'p': vertcat(x0_sym, self.d, self.p)
        }

        # Solver
        self.solver = nlpsol('solver', 'ipopt', self.nlp, opts)  # nlp solver construction

    def optimize_dyn(self, xf, df=[], pf=[], ksim=None):
        """
        Performs 1 optimization step 
        """

        # Solver run
        sol = self.solver(x0=vertcat(*self.w0), p=vertcat(xf, df, pf),
                          lbx=vertcat(*self.lbw), ubx=vertcat(*self.ubw),
                          lbg=vertcat(*self.lbg), ubg=vertcat(*self.ubg))
        flag = self.solver.stats()

        if ksim != None:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if optimization converged
                print('Optimization step ' + str(ksim) + ': Solver did not converge.')
            else:
                print('Optimization step ' + str(ksim) + ': Optimal Solution Found.')
        else:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if optimization converged
                print('Optimization step: Solver did not converge.')
            else:
                print('Optimization step: Optimal Solution Found.')

        # Solution
        wopt = sol['x'].full()  # solution
        self.w0 = copy.deepcopy(wopt)  # solution as guess for the next opt step
        xopt = np.zeros((self.N + 1, self.x.shape[0]))
        uopt = np.zeros((self.N, self.u.shape[0]))
        for i in range(0, self.x.shape[0]):
            xopt[:, i] = wopt[i::self.x.shape[0] + self.u.shape[0] +
                                 self.x.shape[0] * self.m].reshape(-1)  # optimal state
        for i in range(0, self.u.shape[0]):
            uopt[:, i] = wopt[self.x.shape[0] + self.x.shape[0] * self.m + i::self.x.shape[0] +
                                                                              self.x.shape[0] * self.m + self.u.shape[
                                                                                  0]].reshape(-1)  # optimal inputs
        return {
            'x': xopt,
            'u': uopt
        }


class AEKF:
    """
    This class creates an Adaptative Extended Kalman Filter using casadi 
    symbolic framework (regular EKF if there's no theta) 
    """

    def __init__(self, dt, P0, Q, R, x, u, y, dx, theta):
        self.x = x
        self.u = u
        self.y = y
        self.theta = theta
        self.x_ = vertcat(self.x, self.theta)  # extended state vector
        self.Q = Q  # process noise covariance matrix
        self.R = R  # measurement nosie covariance matrix
        self.Pk = copy.deepcopy(P0)  # estimation error covariance matrix

        # Model equations
        dx_ = []
        for i in range(0, self.x.shape[0]):
            dx_.append(x[i] + dt * dx[i])
        for j in range(0, self.theta.shape[0]):
            dx_.append(theta[j])
        self.dx_ = vertcat(*dx_)
        self.F_ = Function('F_EKF', [self.x_, self.u], [self.dx_])  # state equation
        self.JacFx_ = Function('JacFx_EKF', [self.x_, self.u],
                               [jacobian(self.dx_, self.x_)])  # jacobian of F respective to x
        self.H_ = Function('H_EKF', [self.x_, self.u], [self.y])  # output equation
        self.JacHx_ = Function('JacHx_EKF', [self.x_, self.u],
                               [jacobian(self.y, self.x_)])  # jacobian of H respective to x

    def update_state(self, xkhat, uf, ymeas):
        """
        Performs 1 model update step
        """

        Fk = self.JacFx_(xkhat, uf).full()
        xkhat_pri = self.F_(xkhat, uf).full()  # priori estimate of xk
        Pk_pri = Fk @ self.Pk @ Fk.transpose() + self.Q  # priori estimate of Pk
        Hk = self.JacHx_(xkhat_pri, uf).full()
        Kk = (Pk_pri @ Hk.T) @ (np.linalg.inv(Hk @ Pk_pri @ Hk.T + self.R))  # Kalman gain
        xkhat_pos = xkhat_pri + Kk @ ((ymeas - self.H_(xkhat_pri, uf)).full())  # posteriori estimate of xk
        self.Pk_pos = (np.eye(self.x.shape[0] + self.theta.shape[0]) - Kk @ Hk) @ Pk_pri  # posteriori estimate of Pk
        self.Pk = copy.deepcopy(self.Pk_pos)

        # Estimations
        return {
            'x': xkhat_pos[:self.x.shape[0]],
            'theta': xkhat_pos[-self.theta.shape[0]:]
        }


class LSE:
    """
    This class creates a steady-state Least-Squares parameter estimator using 
    casadi symbolic framework 
    """

    def __init__(self, F, R, x, y, u, theta, thetaguess=None, lbtheta=None,
                 ubtheta=None, rootfinder=None, opts={}):
        self.x = x
        self.y = y
        self.u = u
        self.theta = theta
        self.x0 = MX.sym('x0', self.x.shape[0])  # guess for rootfinder
        self.rfsolver = rootfinder('F_SS', 'newton', F) if rootfinder is None \
            else rootfinder  # steady-state model
        J = (self.y - vcat(self.rfsolver(self.x0, self.u, self.theta))
        [:self.x.shape[0]]).T @ R @ (self.y - vcat(self.rfsolver(self.x0,
                                                                 self.u, self.theta))[
                                              :self.x.shape[0]])  # quadratic error cost function

        # Guesses and bounds
        thetaguess = np.zeros(self.theta.shape[0]) if thetaguess is None else thetaguess
        lbtheta = -inf * np.ones(self.theta.shape[0]) if lbtheta is None else lbtheta
        ubtheta = -inf * np.ones(self.theta.shape[0]) if ubtheta is None else ubtheta

        # Empty NLP
        self.w = []
        self.w0 = []
        self.lbw = []
        self.ubw = []

        # Start NLP        
        self.w += [self.theta]  # theta as decision variable
        self.w0 += list(thetaguess)
        self.lbw += list(lbtheta)
        self.ubw += list(ubtheta)

        # NLP
        self.nlp = {
            'x': vertcat(*self.w),
            'f': J,
            'p': vertcat(self.x0, self.u, self.y)
        }

        # Solver
        self.solver = nlpsol('solver', 'ipopt', self.nlp, opts)  # nlp solver construction

    def update_par(self, xf=None, uf=None, ymeas=None, ksim=None):
        """
        Performs 1 model update step
        """

        xf = np.zeros(self.x.shape[0]) if xf is None else xf
        uf = np.zeros(self.u.shape[0]) if uf is None else uf
        ymeas = np.zeros(self.y.shape[0]) if ymeas is None else ymeas

        # Solver run
        sol = self.solver(x0=vertcat(*self.w0), p=vertcat(xf, uf, ymeas),
                          lbx=vertcat(*self.lbw), ubx=vertcat(*self.ubw))
        flag = self.solver.stats()

        if ksim != None:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if optimization converged
                print('Estimation step' + str(ksim) + ': Solver did not converge.')
            else:
                print('Estimation step ' + str(ksim) + ': Optimal Solution Found.')

        else:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if optimization converged
                print('Estimation step: Solver did not converge.')
            else:
                print('Estimation step: Optimal Solution Found.')

        # Solution
        wopt = sol['x'].full()  # estimated parameters
        self.w0 = copy.deepcopy(wopt)  # solution as guess for the next opt step
        return {
            'thetahat': wopt
        }


class NMPC:
    """
    This class creates an NMPC using casadi symbolic framework
    """

    def __init__(self, dt, N, M, Q, W, x, u, c, d, p, dx, R=None, xguess=None,
                 uguess=None, lbx=None, ubx=None, lbu=None, ubu=None, lbdu=None,
                 ubdu=None, tgt=False, disc='collocation', m=3, pol='legendre', 
                 DRTO=False, solver_opts={}):

        self.dt = dt
        self.dx = dx
        self.x = x
        self.c = c
        self.u = u
        self.d = d
        self.p = p
        self.N = N
        self.M = M
        self.disc = disc
        self.m = m
        self.pol = pol
        self.Q = Q
        self.W = W

        # Target matrix R
        R = np.zeros((self.u.shape[0], self.u.shape[0])) if R is None else R

        # Guesses
        xguess = np.zeros(self.x.shape[0]) if xguess is None else xguess
        uguess = np.zeros(self.u.shape[0]) if uguess is None else uguess
        lbx = -inf * np.ones(self.x.shape[0]) if lbx is None else lbx
        lbu = -inf * np.ones(self.u.shape[0]) if lbu is None else lbu
        lbdu = -inf * np.ones(self.u.shape[0]) if lbdu is None else lbdu
        ubx = +inf * np.ones(self.x.shape[0]) if ubx is None else ubx
        ubu = +inf * np.ones(self.u.shape[0]) if ubu is None else ubu
        ubdu = -inf * np.ones(self.u.shape[0]) if ubdu is None else ubdu

        # Removing Nones inside vectors
        if None in xguess: xguess = np.array([0 if v is None else v for v in xguess])
        if None in uguess: uguess = np.array([0 if v is None else v for v in uguess])
        if None in lbx: lbx = np.array([-inf if v is None else v for v in lbx])
        if None in lbu: lbu = np.array([-inf if v is None else v for v in lbu])
        if None in lbdu: lbdu = np.array([-inf if v is None else v for v in lbdu])
        if None in ubx: ubx = np.array([+inf if v is None else v for v in ubx])
        if None in ubu: ubu = np.array([+inf if v is None else v for v in ubu])
        if None in ubdu: ubdu = np.array([-inf if v is None else v for v in ubdu])

        # Quadratic cost function
        self.sp = MX.sym('SP', self.c.shape[0])
        self.target = MX.sym('Target', self.u.shape[0])
        self.uprev = MX.sym('u_prev', self.u.shape[0])

        J = (self.c - self.sp).T @ Q @ (self.c - self.sp) + (self.u - self.target).T \
            @ R @ (self.u - self.target) + (self.u - self.uprev).T @ W @ (self.u - self.uprev)
        self.F = Function('F', [self.x, self.u, self.d, self.p, self.sp, self.target,
                                self.uprev], [self.dx, J], ['x', 'u', 'd', 'p',
                                'sp', 'target', 'u_prev'], ['dx', 'J'])  # NMPC model function

        # Check if the setpoints and targets are trajectories
        if not DRTO:
            spk = self.sp
            targetk = self.target
        else:
            spk = MX.sym('SP_k', 2*(N+1))
            targetk = MX.sym('Target_k', 2*N)

        if not tgt:
            targetk = MX.sym('Target_k', 0)

        # "Lift" initial conditions
        xk = MX.sym('x0', self.x.shape[0])  # first point at each interval
        x0_sym = MX.sym('x0_par', self.x.shape[0])  # first point
        u0_sym = MX.sym('u0_par', self.u.shape[0])
        uk_prev = u0_sym

        # Empty NLP
        self.w = []
        self.w0 = []
        self.lbw = []
        self.ubw = []
        self.g = []
        self.lbg = []
        self.ubg = []
        self.J = 0

        # Discretization
        if self.disc == 'collocation':
            # NLP
            self.w += [xk]
            self.w0 += list(xguess)
            self.lbw += list(lbx)
            self.ubw = list(ubx)
            self.g += [xk - x0_sym]
            self.lbg += list(np.zeros(self.dx.shape[0]))
            self.ubg += list(np.zeros(self.dx.shape[0]))

            # Polynomials
            self.tau = np.array([0] + collocation_points(self.m, self.pol))
            self.L = np.zeros((self.m + 1, 1))
            self.Ldot = np.zeros((self.m + 1, self.m + 1))
            self.Lint = self.L
            for i in range(0, self.m + 1):
                coeff = 1
                for j in range(0, self.m + 1):
                    if j != i:
                        coeff = np.convolve(coeff, [1, -self.tau[j]]) / (self.tau[i] - self.tau[j])
                self.L[i] = np.polyval(coeff, 1)
                ldot = np.polyder(coeff)
                for j in range(0, self.m + 1):
                    self.Ldot[i, j] = np.polyval(ldot, self.tau[j])
                lint = np.polyint(coeff)
                self.Lint[i] = np.polyval(lint, 1)

            # NLP build
            for k in range(0, self.N):
                # State at collocation points
                xki = []
                for i in range(0, self.m):
                    xki.append(MX.sym('x_' + str(k + 1) + '_' + str(i + 1), self.x.shape[0]))
                    self.w += [xki[i]]
                    self.lbw += [lbx]
                    self.ubw += [ubx]
                    self.w0 += [xguess]

                # uk as decision variable
                uk = MX.sym('u_' + str(k + 1), self.u.shape[0])
                self.w += [uk]
                self.lbw += list(lbu)
                self.ubw += list(ubu)
                self.w0 += list(uguess)
                self.g += [uk - uk_prev]  # delta_u

                # Control horizon
                if k >= self.M:
                    self.lbg += list(np.zeros(self.u.shape[0]))
                    self.ubg += list(np.zeros(self.u.shape[0]))
                else:
                    self.lbg += list(lbdu)
                    self.ubg += list(ubdu)

                # Loop over collocation points
                xk_end = self.L[0] * xk
                for i in range(0, self.m):
                    xk_end += self.L[i + 1] * xki[i]  # add contribution to the end state
                    xc = self.Ldot[0, i + 1] * xk  # expression for the state derivative at the collocation point
                    for j in range(0, m):
                        xc += self.Ldot[j + 1, i + 1] * xki[j]
                    if not DRTO:  # check if the setpoints and targets are trajectories
                        fi = self.F(xki[i], uk, self.d,self.p, spk, targetk, uk_prev)
                    else:
                        fi = self.F(xki[i], uk, self.d, self.p, vertcat(spk[k], spk[k+N+1]),
                                    vertcat(targetk[k], targetk[k+N]), uk_prev)
                    self.g += [self.dt * fi[0] - xc]  # model equality contraints reformulated
                    self.lbg += [np.zeros(self.x.shape[0])]
                    self.ubg += [np.zeros(self.x.shape[0])]
                    self.J += self.dt * fi[1] * self.Lint[i + 1]  # add contribution to obj. quadrature function

                # New NLP variable for state at end of interval
                xk = MX.sym('x_' + str(k + 2), self.x.shape[0])
                self.w += [xk]
                self.lbw += list(lbx)
                self.ubw += list(ubx)
                self.w0 += list(xguess)

                # No shooting-gap constraint
                self.g += [xk - xk_end]
                self.lbg += list(np.zeros(self.x.shape[0]))
                self.ubg += list(np.zeros(self.x.shape[0]))

                # u(k-1)
                uk_prev = copy.deepcopy(uk)

        elif self.disc == 'single_shooting':
            # NLP build
            xi = x0_sym
            for k in range(0, self.N):
                uk = MX.sym('u_' + str(k + 1), self.u.shape[0])
                self.w += [uk]
                self.lbw += list(lbu)
                self.ubw += list(ubu)
                self.w0 += list(uguess)
                self.g += [uk - uk_prev]  # delta_u

                # Control horizon
                if k >= self.M:
                    self.lbg += list(np.zeros(self.u.shape[0]))
                    self.ubg += list(np.zeros(self.u.shape[0]))
                else:
                    self.lbg += list(lbdu)
                    self.ubg += list(ubdu)

                # Integrate till the end of the interval
                fi = self.F(xi, uk, self.d, self.p, spk, targetk, uk_prev)
                xi += self.dt*fi[0]
                self.J += fi[1]

                # Inequality constraint
                self.g += [xi]
                self.lbg += list(lbx)
                self.ubg += list(ubx)

                # u(k-1)
                uk_prev = copy.deepcopy(uk)

        # NLP 
        self.nlp = {
            'x': vertcat(*self.w),
            'f': self.J,
            'g': vertcat(*self.g),
            'p': vertcat(x0_sym, u0_sym, self.d, self.p, spk, targetk)
        }  # nlp construction

        # Solver
        self.solver = nlpsol('solver', 'ipopt', self.nlp, solver_opts)  # nlp solver construction

    def calc_actions(self, x0, u0, sp, target=[], d0=[], p0=[], ksim=None):
        """
        Performs 1 optimization step for the NMPC 
        """

        # Solver run
        sol = self.solver(x0=vertcat(*self.w0), p=vertcat(x0, u0, d0, p0, sp, target),
                          lbx=vertcat(*self.lbw), ubx=vertcat(*self.ubw),
                          lbg=vertcat(*self.lbg), ubg=vertcat(*self.ubg))
        flag = self.solver.stats()

        if ksim != None:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if solver converged
                print('Time step ' + str(ksim) + ': NMPC solver did not converge.')
            else:
                print('Time step ' + str(ksim) + ': NMPC optimal solution found.')
        else:
            if flag['return_status'] != 'Solve_Succeeded':  # checks if solver converged
                print('Time step: NMPC solver did not converge.')
            else:
                print('Time step: NMPC optimal solution found.')

        # Solution
        wopt = sol['x'].full()
        self.w0 = copy.deepcopy(wopt)  # solution as guess for the next opt step

        if self.disc == 'collocation':
            xopt = np.zeros((self.N + 1, self.x.shape[0]))
            uopt = np.zeros((self.N, self.u.shape[0]))
            for i in range(0, self.x.shape[0]):
                xopt[:, i] = wopt[i::self.x.shape[0] + self.u.shape[0] +
                                     self.x.shape[0] * self.m].reshape(-1)  # optimal state
            for i in range(0, self.u.shape[0]):
                uopt[:, i] = wopt[self.x.shape[0] + self.x.shape[0] * self.m + i::self.x.shape[0] +
                                                                                  self.x.shape[0] * self.m +
                                                                                  self.u.shape[0]].reshape(
                    -1)  # optimal inputs

            # First control action
            uin = uopt[0, :]
            return {
                'x': xopt,
                'u': uopt,
                'uin': uin
            }
        elif self.disc == 'single_shooting':
            uopt = wopt

            # First control action
            uin = uopt[:self.u.shape[0]]
            return {
                'u': uopt,
                'u_in': uin
            }


class MHE:
    """
      This class creates an MHE using casadi symbolic framework
      """

    def __init__(self, dt, N, x, u, d, p, dx, Q, W=None, R=None, xguess=None,
                 uguess=None, dguess=None, pguess=None, lbx=None, ubx=None,
                 lbu=None, ubu=None, lbd=None, lbp=None, ubd=None, ubp=None,
                 pol='legendre', m=3, solver_opts={}):

        self.dt = dt
        self.dx = dx
        self.x = x
        self.u = u
        self.d = d
        self.p = p
        self.N = N
        self.m = m
        self.pol = pol

        # State estimation
        self.Q = Q
        self.ymeas = MX.sym('y_meas', self.x.shape[0])
        ymeask = MX.sym('y_meas_k', N, self.ymeas.shape[0])
        xguess = self.x.shape[0]*[0] if xguess is None else xguess
        lbx = list(-inf*np.ones(self.x.shape[0])) if lbx is None else lbx
        ubx = list(+inf*np.ones(self.x.shape[0])) if ubx is None else ubx

        # Parameter estimation?
        if R.all:
            self.R = R # parameter matrix
            self.theta = vertcat(self.d, self.p) # disturbances + uncertain parameters
            self.thetaref = MX.sym('theta_ref', self.theta.shape[0]) # reference
            thetarefk = MX.sym('theta_ref_k', N, self.thetaref.shape[0])  # reference vector
            dguess = self.d.shape[0]*[0] if dguess is None else dguess
            pguess = self.p.shape[0]*[0] if pguess is None else pguess
            thetaguess = dguess + pguess
            lbd = list(-inf*np.ones(self.d.shape[0])) if lbd is None else lbd
            lbp = list(-inf*np.ones(self.p.shape[0])) if lbp is None else lbp
            ubd = list(+inf*np.ones(self.d.shape[0])) if ubd is None else ubd
            ubp = list(+inf*np.ones(self.p.shape[0])) if ubp is None else ubp
        else:
            self.R = np.zeros((self.theta.shape[0], self.theta.shape[0]))
            self.theta = MX.sym('theta', 0)
            self.thetaref = MX.sym('theta_ref', 0)
            thetaguess = []
            lbd = []
            ubd = []
            lbp = []
            ubp = []
        lbtheta = lbd + lbp
        ubtheta = ubd + ubp

        # Input estimation?
        if W:
            self.W = W
            self.unom = MX.sym('u_nom', self.u.shape[0])
            unomk = MX.sym('u_nom_k', N, self.unom.shape[0])
            uguess = self.u.shape[0]*[0] if uguess is None else uguess
            lbu = list(-inf*np.ones(self.u.shape[0]) if lbu is None else lbu)
            ubu = list(+inf*np.ones(self.u.shape[0]) if ubu is None else ubu)
        else:
            self.W = np.zeros((self.u.shape[0], self.u.shape[0]))
            self.unom = MX.sym('u_nom', 0)
            uguess = []
            lbu = []
            ubu = []

        # Removing Nones inside vectors
        if None in xguess: xguess = np.array([0 if v is None else v for v in xguess])
        if None in uguess: uguess = np.array([0 if v is None else v for v in uguess])
        if None in thetaguess: thetaguess = np.array([0 if v is None else v for v in thetaguess])
        if None in lbx: lbx = np.array([-inf if v is None else v for v in lbx])
        if None in lbu: lbu = np.array([-inf if v is None else v for v in lbu])
        if None in ubx: ubx = np.array([+inf if v is None else v for v in ubx])
        if None in ubu: ubu = np.array([+inf if v is None else v for v in ubu])

        # Quadratic cost function
        J = (self.x - self.ymeas).T @ self.Q @(self.x - self.ymeas) + \
            (self.theta - self.thetaref).T @ self.R @ (self.theta - self.thetaref)

        # MHE model function
        self.F = Function('F_MHE', [self.x, self.u, self.d, self.p, self.ymeas, self.unom, self.thetaref],
                          [self.dx, J], ['x', 'u', 'd', 'p', 'y_meas', 'u_nom', 'theta_ref'], ['dx', 'J'])

        # "Lift" initial conditions
        xk = MX.sym('x0', self.x.shape[0])  # first state at each interval
        x0_sym = MX.sym('x0_par', self.x.shape[0])  # initial state

        # Empty NLP
        self.w = []
        self.w0 = []
        self.lbw = []
        self.ubw = []
        self.g = []
        self.lbg = []
        self.ubg = []
        self.J = 0

        # NLP
        self.w += [xk]
        self.w0 += xguess
        self.lbw += lbx
        self.ubw = ubx
        self.g += [xk - x0_sym]
        self.lbg += self.dx.shape[0]*[0]
        self.ubg += self.dx.shape[0]*[0]

        # Polynomials
        self.tau = np.array([0] + collocation_points(self.m, self.pol))
        self.L = np.zeros((self.m + 1, 1))
        self.Ldot = np.zeros((self.m + 1, self.m + 1))
        self.Lint = self.L
        for i in range(0, self.m + 1):
            coeff = 1
            for j in range(0, self.m + 1):
                if j != i:
                    coeff = np.convolve(coeff, [1, -self.tau[j]])/(self.tau[i] - self.tau[j])
            self.L[i] = np.polyval(coeff, 1)
            ldot = np.polyder(coeff)
            for j in range(0, self.m + 1):
                self.Ldot[i, j] = np.polyval(ldot, self.tau[j])
            lint = np.polyint(coeff)
            self.Lint[i] = np.polyval(lint, 1)

        # NLP build
        for k in range(0, self.N):
            # State at collocation points
            xki = []
            for i in range(0, self.m):
                xki.append(MX.sym('x_' + str(k + 1) + '_' + str(i + 1), self.x.shape[0]))
                self.w += [xki[i]]
                self.lbw += lbx
                self.ubw += ubx
                self.w0 += xguess

            # uk and thetak as decision variables
            uk = MX.sym('u_' + str(k + 1), self.unom.shape[0])
            thetak = MX.sym('theta_k' + str(k + 1), self.thetaref.shape[0])
            self.w += [uk, thetak]
            self.lbw += lbu + lbtheta
            self.ubw += ubu + ubtheta
            self.w0 += uguess + thetaguess

            # Loop over collocation points
            xk_end = self.L[0]*xk
            for i in range(0, self.m):
                xk_end += self.L[i + 1]*xki[i]  # add contribution to the end state
                xc = self.Ldot[0, i + 1]*xk  # expression for the state derivative at the collocation point
                for j in range(0, m):
                    xc += self.Ldot[j + 1, i + 1]*xki[j]
                    fi = self.F(xki[i], uk, self.d, self.p, ymeask[k, :], thetarefk[k, :])
                self.g += [self.dt*fi[0] - xc]  # model equality constraints reformulated
                self.lbg += self.x.shape[0]*[0]
                self.ubg += self.x.shape[0]*[0]
                self.J += self.dt*fi[1]*self.Lint[i + 1]  # add contribution to obj. quadrature function

            # New NLP variable for state at end of interval
            xk = MX.sym('x_' + str(k + 2), self.x.shape[0])
            self.w += [xk]
            self.lbw += lbx
            self.ubw += ubx
            self.w0 += xguess

            # No shooting-gap constraint
            self.g += [xk - xk_end]
            self.lbg += self.x.shape[0] * [0]
            self.ubg += self.x.shape[0] * [0]

        # NLP construction
        # NLP parameters
        if self.R and self.W:
            par = vertcat(x0_sym, ymeask, unomk, thetarefk)
        elif self.R and not self.W:
            par = vertcat(x0_sym, self.u, ymeask, thetarefk)
        elif not self.R and self.W:
            par = vertcat(x0_sym, self.d, self.p, ymeask, unomk)
        else:
            par = vertcat(x0_sym, self.u, self.d, self.p, ymeask)

        # Dict
        self.nlp = {
            'x': vertcat(*self.w),
            'f': self.J,
            'g': vertcat(*self.g),
            'p': par
        }

        # Solver
        self.solver = nlpsol('solver', 'ipopt', self.nlp, solver_opts)  # nlp solver construction

    def update(self, x0, ymeas, uf=[], df=[], pf=[], unom=[], thetaref=[], ksim=None):
        """
        Performs 1 estimation step for the MHE
        """

        # Solver parameters
        if self.R and self.W:
            par = vertcat(x0, ymeas, unom, thetaref)
        elif self.R and not self.W:
            par = vertcat(x0, uf, ymeas, thetaref)
        elif not self.R and self.W:
            par = vertcat(x0, df, pf, ymeas, unom)
        else:
            par = vertcat(x0, uf, df, pf, ymeas)

        # Solver run
        sol = self.solver(x0=vertcat(*self.w0), p=par,
                          lbx=vertcat(*self.lbw), ubx=vertcat(*self.ubw),
                          lbg=vertcat(*self.lbg), ubg=vertcat(*self.ubg))
        flag = self.solver.stats()

        # Check convergence
        if ksim != None:
            if flag['return_status'] != 'Solve_Succeeded':
                print('Time step ' + str(ksim) + ': MHE solver did not converge.')
            else:
                print('Time step ' + str(ksim) + ': MHE optimal solution found.')
        else:
            if flag['return_status'] != 'Solve_Succeeded':
                print('Time step: MHE solver did not converge.')
            else:
                print('Time step: MHE optimal solution found.')

        # Solution
        wopt = sol['x'].full()

        # Solution as guess for the next opt step
        self.w0 = copy.deepcopy(wopt)

        # Optimal states
        xopt = np.zeros((self.N + 1, self.x.shape[0]))
        for i in range(0, self.x.shape[0]):
            xopt[:, i] = wopt[i::
                              self.x.shape[0] +
                              self.u.shape[0] +
                              self.theta.shape[0] +
                              self.x.shape[0]*self.m].reshape(-1)
        # Optimal inputs
        uopt = np.zeros((self.N, self.u.shape[0])) if self.W else None
        for i in range(0, self.unom.shape[0]):
            uopt[:, i] = wopt[self.x.shape[0] +
                              self.x.shape[0]*self.m +
                              i::
                              self.x.shape[0] +
                              self.x.shape[0]*self.m +
                              self.u.shape[0]].reshape(-1)

        # Optimal parameters
        thetaopt = np.zeros((self.N, self.theta.shape[0])) if self.R else None
        for i in range(0, self.thetaref.shape[0]):
            thetaopt[:, i] = wopt[self.x.shape[0] +
                                  self.x.shape[0]*self.m +
                                  self.u.shape[0] +
                                  i::
                                  self.x.shape[0] +
                                  self.x.shape[0]*self.m +
                                  self.u.shape[0] +
                                  self.theta.shape[0]].reshape(-1)

            # Estimates
            xhat = xopt[-1, :]
            uhat = uopt[-1, :]
            thetahat = thetaopt[-1, :]

            return {
                'x': xopt,
                'u': uopt,
                'theta': thetaopt,
                'x_hat': xhat,
                'u_hat': uhat,
                'theta_hat': thetahat
            }

In [58]:
# Model parameters for the Van de Vusse CSTR: 4 states and 2 inputs

from casadi import *

# Sampling time 
dt = 0.1/40

# Parameters 
#k10 = 1.287e12 #1st reaction frequency factor (h-1)
k20 = 1.287e12 #2nd reaction frequency factor (h-1)
k30 = 9.043e9 #3rd reaction frequency factor (L/mol L)
E1 = 9758.3  #1st reaction activation energy /R (K)
E2 = 9758.3 #2nd reaction activation energy /R (K)
E3 = 8560 #3rd reaction activation energy /R (K)
deltaH1 = 4.2  #1st reaction enthalpy (kJ/mol)
deltaH2 = -11 #2nd reaction enthalpy (kJ/mol)
deltaH3 = -41.85 #3rd reaction enthalpy (kJ/mol)
rho = 0.9342 #density (kg/L)
cp = 3.01 #heat capacit (kJ/kg K)
Ar = 0.215 #jacket area (m2)
Kw = 4032 #jacket heat transfer coefficient (kJ/h m2 K)
V = 10 #reactor volume (L)
mk = 5;
#cpk = 2;

# States
Ca = MX.sym('C_A',1) #yield of A (mol/L) 
Cb = MX.sym('C_B',1) #yield of B (mol/L)
T = MX.sym('T',1) #system temperature (C)
Tk = MX. sym('T_k',1) #jacket temperature (C)
x = vertcat(Ca, Cb, T, Tk)
c = vertcat(Cb, T) #controlled variables

# Outputs
y = vertcat(Ca, Cb, T, Tk)

# Inputs
f = MX.sym('F/V', 1) #spacial velocity (h-1)
Qk = MX. sym('Q_k', 1) #jacket heat (kJ/h)
u = vertcat(f, Qk)

# Disturances
Cain = MX.sym('C_Ain',1) #inlet yield of A (unmeasured)
Tin = MX.sym('T_in',1)  #inlet temperature (measured)
d = vertcat(Cain, Tin)

# Uncertain parameters
k10 = MX.sym('k10', 1)
cp = MX.sym('cp', 1)
p = vertcat(k10, cp)

# ODE system
dCadt = f*(Cain - Ca) - Ca*k10*exp(-E1/(T + 273.15)) - Ca**2*k30*exp(-E3/(T + 273.15))
dCbdt = -f*Cb + Ca*k10*exp(-E1/(T + 273.15)) - Cb*k20*exp(-E2/(T + 273.15))
dTdt = f*(Tin - T) + (Kw*Ar*(Tk - T)/V + (Ca*(-deltaH1)*k10*exp(-E1/(T + 273.15))) + \
       (Cb*(-deltaH2)*k20*exp(-E2/(T + 273.15))) + (Ca**2*(-deltaH3)*k30*exp(-E3/(T + 273.15))))/(rho*cp)
dTkdt = (Qk + Kw*Ar*(T - Tk))/mk/cp;
dx = vertcat(dCadt, dCbdt, dTdt, dTkdt)

# Cost function
J = - (Cb/(Cain-Ca) + Cb/Cain - 5e-4*Tk)

In [59]:
# from VdV4x2 import *
# from CasadiTools import *
from progress.bar import IncrementalBar
import matplotlib.pyplot as plt
import time
import math

# Process
process = ODEModel(dt=dt, x=x, y=y, u=u, dx=dx, d=d, p=p) #process object
process.get_equations(intg='idas')

# SS optimizer opts
opts = {
    'warn_initial_bounds': False, 'print_time': False, 
    'ipopt': {'print_level': 1}
    }

# Initial guesses
Caguess = 1.7949
Cbguess = 1.0787
Tguess = 144.2363
fguess = 100
Tkguess = 150
Qkguess = -4000
Cainguess = 5
Tinguess = 130
k01guess = 1.287e12
cpguess = 2
xguess = [Caguess, Cbguess, Tguess, Tkguess]
uguess = [fguess, Qkguess]
dguess = [Cainguess, Tinguess]
pguess = [k01guess, cpguess]

# Bounds
lbCa = 0.1
ubCa = 5
lbCb = 0.1
ubCb = 2
lbT = 30
ubT = 200
lbTk = 30
ubTk = 200
lbf = 10
ubf = 400
lbQk = -8500
ubQk = 0
lbCain = 0.1
ubCain = 6
lbTin = 30
ubTin = 200
lbk01 = .5*k01guess
ubk01 = 1.5*k01guess
lbcp = .5*cpguess
ubcp = 1.5*cpguess
lbx = [lbCa, lbCb, lbT, lbTk]
ubx = [ubCa, ubCb, ubT, ubTk]
lbu = [lbf, lbQk]
ubu = [ubf, ubQk]
lbd = [lbCain, lbTin]
ubd = [ubCain, ubTin]
lbp = [lbk01, lbcp]
ubp = [ubk01, ubcp]

# MHE
Q = np.diag([1e1, 1e1, 1e2, 1e2])*1e-4
R = np.diag([3e-2, 5e-3, 8e-1, 5e-3])
N = 40

mhe = MHE(dt=dt, N=N, Q=Q, R=R, x=x, u=u, d=d, p=p, dx=dx, xguess=xguess,
          uguess=uguess, dguess=dguess, pguess=pguess, lbx=lbx, ubx=ubx,
          lbu=lbu, ubu=ubu, lbd=lbd, ubd=ubd, lbp=lbp, ubp=ubp)

# NMPC
N = 40
M = 10
Q = np.diag([1, 1e-3])
W = np.diag([1e-5, 1e-6])
lbdf = -50
ubdf = 50
lbdQk = -50
ubdQk = 50
lbdu = [lbdf, lbdQk]
ubdu = [ubdf, ubdQk]

nmpc = NMPC(dt=dt, N=N, M=M, Q=Q, W=W, x=x, u=u, c=c, d=d, p=p, dx=dx,
       xguess=xguess, uguess=uguess, lbx=lbx, ubx=ubx, lbu=lbu, ubu=ubu, 
       lbdu=lbdu, ubdu=ubdu, disc='single_shooting')

# Initialization
t = 1 #counter
tsim = 2 #h
niter = math.ceil(tsim/dt)

xf = [3.08275401, 0.52532486, 122.27127671, 77.75680223]
uf = [120.04167236,  -4000]
dist = [4, 130]
par_model = [1.287e12, 3.01]
par_plant = [1.287e12*.95, 3.01*0.8]
spsim = [0.5, 120]
xhat = copy.deepcopy(xf)
dhat = copy.deepcopy(dist[0])
phat = copy.deepcopy(par_model)
ysim = np.zeros([niter, 4])
usim = np.zeros([niter, 2])
dsim = np.zeros([niter, 2])
psim = np.zeros([niter, 2])
xest = np.zeros([niter, 4])
dest = np.zeros([niter, 1])
pest = np.zeros([niter, 2])
ymeassim = np.zeros([niter, 4])
d1meassim = np.zeros([niter, 1])
d2hat = dhat[1]*N
phat = phat*N
cpu_time = []
bar = IncrementalBar('Simulation in progress', max=niter) #progress bar

# Simulation 
for ksim in range(0, niter):
    start = time.time() #comp time
    n = ksim/niter

    # Disturbances
    if n > 1/4 and n < 2/4:
        dist = [5.1, 130]
    elif n >= 2/4:
        dist = [5.1, 130*1.1]
    else:
        dist = [4, 130]

    # Plant
    sim = process.simulate_step(xf=xf, uf=uf, df=dist, pf=par_plant)
    ymeas = sim['x']*(1 + 0.001*np.random.normal(0, 1))
    d1meas = sim['d'][1]*(1 + 0.001*np.random.normal(0, 1))
    xf = sim['x'].ravel()
    ysim[t-1, :] = xf
    usim[t-1, :] = sim['u']
    dsim[t-1, :] = sim['d']
    psim[t-1, :] = sim['p']
    ymeassim[t-1,:] = ymeas
    d1meassim[t-1] = d1meas

    # MHE
    est = mhe.update(ksim=ksim+1, x0=xhat, uf=uf, ymeas=ymeassim[-N, :],
                     thetaref=vertcat(d1meassim[-N, :], d2hat[-N, :], p1hat, p2hat))

    d1hat = list(est['theta_opt'][0::4])
    d2hat = list(est['theta_opt'][1::4])
    p1hat = list(est['theta_opt'][2::4])
    p2hat = list(est['theta_opt'][3::4])
    d1hat_ = list(d1hat[-1])
    d2hat_ = list(d2hat[-1])
    p1hat_ = list(p1hat[-1])
    p2hat_ = list(p2hat[-1])
    xest[t-1, :] = est['x_hat']
    thetaest[t-1, :] = est['theta_hat']

    # NMPC
    ctrl = nmpc.calc_actions(ksim=ksim+1, x0=xhat, u0=uf, sp=spsim, 
                            d0=d1hat_+d2hat_, p0=p1hat_+p2hat_)
    uf = ctrl['uin']
    t += 1
    end = time.time()
    cpu_time += [end - start]
    bar.next() 

bar.finish()
avg_time = np.mean(cpu_time) # avg time spent at each opt cycle
    
# Plot 
time = np.linspace(0, tsim, niter)

fig1, ax1 = plt.subplots(2, 2, frameon=False) #x 
ax1[0, 0].plot(time, ysim[:, 0], label='Plant') #Ca
ax1[0, 0].plot(time, xest[:, 0], linestyle=None, marker='o', label='MHE')
ax1[0, 0].set_ylabel('C_A [mol/L]')
ax1[0, 0].legend()
ax1[1, 0].plot(time, ysim[:, 1], label='Plant') #Cb 
ax1[1, 0].plot(time, spsim[:, 0], linestyle='--', label='SP') 
ax1[1, 0].plot(time, xest[:, 1], linestyle=None, marker='o', label='MHE')
ax1[1, 0].set_ylabel('C_B [mol/L]')
ax1[1, 0].set_xlabel('time [h]')
ax1[1, 0].legend()
ax1[0, 1].plot(time, ysim[:, 2], label='Plant') #T 
ax1[0, 1].plot(time, spsim[:, 1], linestyle='--', label='SP') 
ax1[0, 1].plot(time, xest[:, 2], linestyle=None, marker='o', label='MHE')
ax1[0, 1].set_ylabel('T [\xb0C]')
ax1[0, 1].legend()
ax1[1, 1].plot(time, ysim[:, 3], label='Plant') #Tk 
ax1[1, 1].plot(time, xest[:, 3], linestyle=None, marker='o', label='MHE')
ax1[1, 1].set_ylabel('T_k [\xb0C]')
ax1[1, 1].legend()

fig2, ax2 = plt.subplots(2, 1, frameon=False) #u
ax2[0].step(time, usim[:, 0]) #f
ax2[0].set_ylabel('f [h^{-1}]')
ax2[0].legend()
ax2[1].step(time, usim[:, 1]) #Qk
ax2[1].set_ylabel('Q_k/(Kw Ar) [kJ/h]')
ax2[1].set_xlabel('time [h]')
ax2[1].legend()
#ax2.legend()

fig3, ax3 = plt.subplots(2, 1, frameon=False) #d
ax3[0].step(time, dsim[:, 0], label='Plant') #Cain
ax3[0].plot(time, dest[:, 0], linestyle=None, marker='o', label='MHE')
ax3[0].set_ylabel('C_{Ain} [mol/h]')
ax3[0].legend()
ax3[1].step(time, dsim[:, 1], label='Plant') #Tin
ax3[1].plot(time, dest[:, 1], linestyle=None, marker='o', label='MHE')
ax3[1].set_ylabel('T_{in} [\xb0C]')
ax3[1].set_xlabel('time [h]')
ax3[1].legend()

fig4, ax4 = plt.subplots(2, 1, frameon=False) #p
ax4[0].step(time, psim[:, 0], label='Plant') #k1
ax4[0].plot(time, pest[:, 0], linestyle=None, marker='o', label='MHE')
ax4[0].set_ylabel('k_1 [h^{-1}]')
ax4[0].legend()
ax4[1].step(time, psim[:, 1], label='Plant') #cp
ax4[1].plot(time, pest[:, 1], linestyle=None, marker='o', label='MHE')
ax4[1].set_ylabel('c_p [kJ/kg K]')
ax4[1].set_xlabel('time [h]')
ax4[1].legend()

plt.show()


RuntimeError: ignored